# Jonathan Halverson
# Thursday, April 28, 2016
# Notes on Spark

Spark is a big data processing engine with many advantages over Hadoop MapReduce. For instance, it is fast, has rich APIs and can be used for interactive SQL queries, streaming, machine learning and graph processing.

Parallel data analysis has become commonplace. Spark offers three main benefits over MapReduce: (1) It is easy to use allowing one to create applications on a laptop using a high level API, (2) Spark is fast thanks to in-memory computing and (3) it is a general engine allowing for different tasks whereas Hadoop has an entire ecosystem of heterogeneous engines.

### Chapter 1

Spark is a cluster computing platform designed to be fast and general-purpose. It is highly accessible offering APIs in Java, Scala and Python. Spark can access any Hadoop data source like Cassandra. Spark comes with a unified stack which means improvements to the core benefit all. It is also easier to maintain.

Spark Core contains components for scheduling, memory manangement, fault recovery and interacting with storage systems.

Spark SQL can be used to work with structured data. Hive Query Language or HQL can also be used. One may mix Spark SQL with various data manipulations. Spark SQL is a replacement to Shark.

Spark Streaming allows for the processing of live data streams. The streaming API closely matches that of the RDD API.

GraphX a is library for manipulating graphs like a social network graph. GraphX extends the Spark RDD API.

A cluster manager is needed to run Spark. It was designed to scale efficiently from one to thousands of compute nodes. Cluster manager choices are Hadoop YARN, Apache Mesos and the Standalone Scheduler.

Data wrangling is transforming data into a particular format so that it can be analyzed. Spark offers support to call out to external programs like Matlab and R.

Software engineers can wrote code to process data and not worry about how the data is stored or about how the analysis is parallelized. They have the ability to monitor, inspect and tune their applications.

The store layers for Spark include any file in HDFS, Amazon S3, Cassandra, Hive and HBase.

### Chapter 2

Spark is written in Scala and runs on the Java Virtual Machine. Why not write it in C++ for the speed advantages? Spark requires Java 6 or higher. Java 8 supports lambda expressions. On a laptop, Spark runs in local mode.

Spark provides different shells for interactive use. Parallel computations can be carried out from the shell on distributed clusters.

In Spark, we express our computation through operations on distributed collections that are automatically parallelized across the cluster. These collections are resilient distributed datasets or RDDS. The driver program was the Spark shell for the intro example. A SparkContest represents a connection to a computing cluster. In the shell the variable sc is automatically created and it is a SparkContext. A SparkContext can be used to build RDDs which can be data from some source (textFile) or data generated in the shell (parallelize).

Lauch the Python shell with: pyspark

Run flat Python scripts: spark-submit script.py

### Chapter 3

An RDD is a distributed collections of elements. In Spark, all work is either creating RDDs, transforming RDDs or calling operations on them to compute a result. The beauty of Spark is that is automatically parallelized the RDDs as well as the operations that act on them. An RDD is immmutable. RDDs are split into multiple partitions which are computed on different nodes of the cluster. RDDs may contain Python objects and classes (presumbly use the parallelize method).

One may apply transformations and actions to RDDs. Transformations make new RDDs like using filter. Actions compute a result based on an RDD such as the first method. Spark computes RDD is lazy manner meaning it doesn't actually evaluate the RDD until an action is called on it. So with:

lines = sc.textFile('text_file')
pythonlines = lines.filter(lambda line: 'Python' in line)

The RDD lines is never computed but only pythonlines. This is because all the the lines in the file (which may be TB in size) are never needed.

One may use the persist method to cause and RDD to remain. Otherwise, it is recompute each time it is needed. RDDs are called resilient because if a partition fails, the lossed data is recomputed elsewhere. Cache() is the same as persist() with the default storage method.

The two ways to create an RDD are to load external data or parallelize a collection in the driver program. The later is not widely used since it requires have all your data initially on one machine. Transformations return RDDs whereas actions return something else. Actions include writing out data or the contents of RDDs. Actions force RDD to be evaluated.

textFile returns an RDD of strings. To turn off unicode use: use_unicode=False

As new RDDs are created from transformations, Spark create a lineage graph which it uses to create each RDD on demand and deal with faults.

Python functions can be passed into actions either by local definition or top-level functions or lambda functions. Becareful not to pass an object that contains references because Spark will send the entire object to the worker nodes. This can cause the code to fail. The idea is to prevent inadvertent serialization.

The map method applies a function to each element. The filter method returns on the True lines. The method flatMap can be used when the applied function returns more that one item like a list or tuple. It combines all the results down to a single sequence.

Sparks offer pseudo set operations like distinct, union, intersection and subtract. The RDDs must be of the same type. Some of these operations are expensive since they require shuffling which involves transferring all the data between the compute nodes to ensure that only one item is returned in the case of distinct.

The take method tries to minimize the number of partitions that it accesses so it represents a biased sample. Take and collect are mainly used for debugging and testing since they can be expensive. The order of the elements can be preserved by using top, which allows for a comparison function.

The foreach method applies a function to all elements of the RDD without bringing them back locally. Useful for posting JSON to a webserver or adding entries to a database.

Cached RDDs using persist() are serialized and stored as pickled objects.

### Chapter 4

The (key, value) pair is the subject. Partitioning or how the elements of an RDD are distributed is also discussed. Communication between the worker nodes can be minimized by proper partitioning. Spark provides special methods for working with pair RDDs.

One still must be careful about passing references into functions for pair RDDs as for RDDs.

Use repartition and coalesce to change the partitioning of the RDD.

Additional actions on RDDs include countByKey(), collectAsMap() and lookup(key).

Partitioning is useful when a RDD is scanned more than once. This is especially true for key-intensive operations such as joins where if the elements with the same keys between the two RDDs are on the same partition then no shuffling across the network is needed. One may use a hash or range partitiioner as well as custom partitioners. There are rules which determine the partitioner of the resulting RDD from a map transformation, for instance.

### Chapter 5

Loading and saving data in different formats and from databases. Spark can work with data stored on local and distributed file systems like NFS, HDFS and GPFS. A variety of file formats are possible such as text, JSON, CSV, TSV, SequenceFiles, etc. Data compression is also possible. Certain compressed files may be read directly. Hadoops new and old file API's can be used.

Spark SQL allows one to read in a table into rows and refernce fields within each row. This has a better API for working with JSON and Hive. There are third-party libraries for connecting to Cassandra, HBase and ElasticSearch as well as JDBC databases.

Working with multiline JSON files is difficult. It is best to read in JSON as a textfile and then convert using a parser.

Sequence files are a popular Hadoop format composed of flat files with key/value pairs. Sometimes the key is set to null. Spark can read sequence files in parallel. These are split-table formats. Serialization is needed to work with data structures not known to Hadoop.

Spark's native file readers are sc.textFile and sc.SequenceFile. Both of these can support the reading of compressed data. To use sc.textFile, the file must be on all nodes on the cluster. NPS, HDFS, APS and GPFS are network filesystems that allow access to a user's file from any node. One can also load the data locally and then use parallelize, but this can be slow. It's best to be on a shared filesystem.

AWS S3 and HDFS are good choices.

Spark SQL is popular for working with structured and semi-structured data. A schema is a consistent set of fields used to described structured data. Spark SQL can work with multiple data sources and include only certain fields if desired. We run a SQL query on the data source and get back an RDD of rows. We can extract the elements of reach row.

Hive is used for storing structured data. Spark SQL can work with it natively. HQL may be written to create RDDs based on tables that exist in Hive. A Hive context object must first be created.

Spark can interoperate with several different databases using either Hadoop connectors or custom Spark versions. Spark can load data from any relational database that supports JDBC including MySQL, Postgres and others. The export feature of the database can be used to work with a static text file instead of the live version of the DB. The DB must be able to handle the load of parallel reads coming from Spark. Spark can interoperate with Cassandra but it has less native support than other databases.

Spark can access HBase through its Hadoop input format. One first makes an HBase configuration and then sc.newAPIHadoopRDD().

ElasticSearch is a new Lucene-based search system. Spark can read and write data from ElasticSearch using ElasticSearch-Hadoop.

### Chapter 6

Spark offers two types of shared variables: accumulators and broadcast variables. The first can be used for debuggin purposes and the later for efficiency in transfer large data structures. Batch operations on a per-partition basis are also introduced. Operations like connecting to a database or downloading a file can be done once per partition as opposed to for every element in the RDD. External libraries may be used if Spark can interact with them through standard input and output. This sounds slow.

Accumulators allow one to aggregate values from worker nodes back to the driver program. A good use is to counter the number of bad lines in an input file as it is read in. This can then be reported back to the driver program which can decide how to proceed.

With small data one can simply examine the data set on the master node. However with large data we cannot do this so accumulators serve an important role. The only operation permissible is +=. However, custom methods exists for different operations. One may also use doubles and floats. Accumulators are write-only meaning worker nodes cannon access their value.

Because of the way Spark handles fault tolerance with accumulators, they should only be used for debugging purposes and not for production.

For operations like  loading a database, use mapPartitions which is called with an iterator to the elements in the partition and returns an iterator of the return elements.

One can also pipe() to external programs and libraries like R. There are also methods and the StatsCounter object for computing descriptive statistics.

### Chapter 7

Cluster mode versus local mode. The three cluster managers are Hadoop YARN, Apache Mesos and the Standalone Scheduler. This chapter is about running a distributed Spark program. The driver communicates with the workers or executors. The driver and the workers each run in their own Java process. A task is the smallest element that are carried out on the worker nodes. In local mode, the driver and worker run in the same process.

spark-submit is the script to be used to submit applications to a cluster. The driver program operates outside of the cluster manager. The cluster manager lauches executors based on the instructions within the main().

Spark comes with a built-in script to run on Amazon EC2 (see spark-ec2). It allows one to use HDFS.

### Chapter 9

Spark SQL is an interface to working with structured and semi-structured data. Structured data is any data that has a schema or a set of fields that describe the data. Spark SQL introduces the DataFrame which is an RDD of Row objects. Hive is the Hadoop SQL engine. Spark SQL does the queries in parallel on distributed tables so it is fast. Spark SQL has dependencies. For example, Hive should be included with the binary. HiveQL is the recommended way of using Spark.

One can use RDD operations on a DataFrame. One can also query existing Hive tables. One needs to register a DataFrame as a TempTable in order to run queries. Use the show, select, filter and groupBy methods to operate on DataFrames. Spark SQL can work with JSON, CSV, Parquet, Hive and so on. Parquet is like Vertica in that they store data in columns.

JDBC is the Java database connector which allows for tools like Tableau to connect with a Spark cluster through its API. The JDBC server is started by running start-thriftserver.sh; it runs in the background. The Beeline client is used to interact with Hive tables using HiveQL.

### Chapter 11

Machine learning in Spark is done with MLlib. It works on RDD so the data is partitioned between the worker nodes. Some models that do not work in parallel are not available in MLlib. One may use a single-node library to train a large number of models on individual nodes (e.g., scikit-learn). MLlib only provides a lower level API. For instance, it does not provide the train-test split method. It does allow for pipelining.

MLlib depends on gfortran and numpy. Of course, the key to machine learning is finding the best features. The library introduces are few data types: vector, labeledpoint, rating and model.

MLlib works on RDDs of either Vectors, LabeledPoint or Ratings. The RDD's are typically constructed via transformations on external data either by reading in a file or running a Spark SQL command. Then apply a map to convert your RDD into the correct input for MLlib.

Vectors can be either dense or sparse. Simply pass a numpy array to create one or a dictionary for sparse vectors.